# SQL Data Exploration - COVID19 Project 

Using publically available data from https://ourworldindata.org/covid-deaths 

## Stage 1 - Database Normalisation
Having downloaded the data, the large table is split into 2 smaller tables and saved as csv files. ___Note:the date format has to be year-month-day___

__Table 1:__ _Death data:_ Move the population column next to the date - remove the vacc stuff (everything from AA onwards). Save as new... This avoids having to do a join on every query. Save as CovidDeaths

__Table 2:__ _Vaccination data:_ On full table delete everything from Z to E. 

## Stage 2 - Uploading to Azure
This is different using the docker - to do this I have installed the following extension: SQL Server Import

1. Right click the database and import wizard - select the server. 
2. Select the CSV file to import - make sure the location is set to the correct database from the default of master.
3. Look at the variable types and allow null values. If these are not correct then the table will not be imported. It will tell you which rows and why so you can alter to float, text, varchar etc. (this is where correct date format comes in).

## Stage 3 - Data Exploration

#### Some simple Data Exploration within the CovidDeath table
When aggregating you neede a group by function
e.g.

In [ ]:
-- Showing countries with highest death rate
SELECT location, MAX(total_deaths) as totaldeathcount 
FROM COVID_PORTFOLIO_PROJECT.dbo.CovidDeaths
GROUP BY LOCATION
ORDER BY totaldeathcount DESC

If the data has the wrong variable type (e.g. varchar when it shoud be int) you can cast: 
_note: I have done this when importing the csv using the import wizard so my values are set as 'float'_ 

In [ ]:
-- Showing countries with highest death rate
SELECT location, MAX(cast(total_deaths as INT)) as totaldeathcount 
FROM COVID_PORTFOLIO_PROJECT.dbo.CovidDeaths
GROUP BY LOCATION
ORDER BY totaldeathcount DESC

When looking through the data we can see that there are continents listed as locations - i.e. their location is set as a continent rather than a country. This means that grouping the locations will show continent summaries too which we don't always want... By using the WHERE clause we can remove these from the queries:

In [ ]:
-- Showing countries with highest death rate
SELECT location, MAX(cast(total_deaths as INT)) as totaldeathcount 
FROM COVID_PORTFOLIO_PROJECT.dbo.CovidDeaths
WHERE continent is not NULL
GROUP BY LOCATION
ORDER BY totaldeathcount DESC

To group by date - you need to use aggregate functions on the other variables. 

In [ ]:
SELECT date, SUM(new_cases) as totalcases, SUM(new_deaths) as totaldeaths, (SUM(new_deaths)/SUM(new_cases))*100 as Deathpercentage -- sums new cases for each date (i.e. per day) and the new deaths to give totals
FROM COVID_PORTFOLIO_PROJECT.dbo.CovidDeaths
WHERE continent is not NULL
GROUP BY date
ORDER BY 1,2

#### Joining the tables: 
The tables can be joined on the location and date. Note that the join requires you to specify which table's variable you are using 
i.e. in this case the CovidDeaths table is renamed as dea and the select command calls the dea.continent etc. This is required throughout and hence why the renaming of the table saves time when typing out.

In [ ]:
-- Looking at total population vs vaccinations
select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations 
FROM covid_portfolio_project.dbo.CovidDeaths as dea JOIN covid_portfolio_project.dbo.CovidVaccinations as vac
    ON dea.LOCATION = vac.location
    and dea.DATE = vac.date
    WHERE dea.continent is not NULL
order by 1, 2

#### Partition By

Using the PARTITION BY allows us to select just one column we want to use our aggregate function on (whereas a GROUP BY would roll them all up into 1)

In [ ]:
-- Looking at total population vs vaccinations
select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
,SUM(vac.new_vaccinations) OVER (PARTITION by dea.location order by dea.LOCATION, dea.date) 
FROM covid_portfolio_project.dbo.CovidDeaths as dea JOIN covid_portfolio_project.dbo.CovidVaccinations as vac
    ON dea.LOCATION = vac.location
    and dea.DATE = vac.date
    WHERE dea.continent is not NULL
order by 2,3

#### CTE to create a rolling count of people vaccinated 

In [ ]:
-- Use CTE
With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
as
(
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_portfolio_project..CovidDeaths dea
Join covid_portfolio_project..CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 
)
Select *, (RollingPeopleVaccinated/Population)*100
From PopvsVac

#### Temp table to create a rolling count of people vaccinated

In [ ]:
-- Temp Table
DROP TABLE if exists #percentpopulationvaccinated
CREATE TABLE #percentpopulationvaccinated
(
continent nvarchar(255),
Location nvarchar(255),
date datetime, 
population numeric,
new_vaccinations numeric, 
rollingpeoplevaccinated numeric
)

INSERT INTO #percentpopulationvaccinated
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(vac.new_vaccinations) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_portfolio_project..CovidDeaths dea
Join covid_portfolio_project..CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 

Select *, (rollingpeoplevaccinated/population)*100 as percentofpeoplevaccinated
FROM #percentpopulationvaccinated